In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point
import requests
import numpy as np
import math
from shapely.ops import nearest_points

In [27]:
stops = gpd.read_file("./output/stops_no_radius/stops_no_radius.shp")
mbta_stops = gpd.read_file('./data/stops_routes/mbtabus/MBTABUSSTOPS_PT.shp')
baseline_map = pd.read_csv('./output/baseline_map_info.csv')

In [7]:
stops.head()

,STOP_ID,impacted_t,proportion,income,income_lev,impacted_p,geometry
0,3077,"416400,100800,100601,100700,100603","0.01,0.17,0.13,0.99,0.29",86562,2,1046,POINT (237120.669 892643.408)
1,841,"130402,130300,130200,130100,130404","0.08,0.26,0.63,0.13,0.02",107359,2,632,POINT (227915.195 892644.017)
2,446,"100500,100400,100300,101102,101101,101002,1009...","0.17,0.18,1.00,0.41,0.06,0.24,0.02,1.00,0.37",46390,2,3056,POINT (234385.661 892644.944)
3,847,"130402,130300,130200,130100,130404","0.08,0.26,0.63,0.13,0.02",107359,2,632,POINT (227912.601 892650.156)
4,3079,"416400,100800,100601,100500,100400,100700,100603","0.00,0.51,0.32,0.01,0.03,0.67,0.06",83532,2,1647,POINT (236644.812 892651.990)


In [6]:
mbta_stops.head()

,STOP_ID,STOP_NAME,TOWN,TOWN_ID,geometry
0,3077,Gallivan Blvd @ opp Marsh St,BOSTON,35,POINT (237120.669 892643.408)
1,841,Lagrange St @ Virgil Rd,BOSTON,35,POINT (227915.195 892644.017)
2,446,Norfolk St @ Nelson St,BOSTON,35,POINT (234385.661 892644.944)
3,847,Lagrange St opp Virgil St,BOSTON,35,POINT (227912.601 892650.156)
4,3079,Adams St @ Minot St,BOSTON,35,POINT (236644.812 892651.990)


In [28]:
baseline_map.drop(columns=['income', 'income_level', 'impacted_pop', 'total_employed', 'revenue_annual','Unnamed: 0'], inplace=True)
baseline_map.head()

,STOP_ID,STOP_NAME,TRACT_ID,route_ids
0,3077,Gallivan Blvd @ opp Marsh St,100700,201
1,841,Lagrange St @ Virgil Rd,130200,"37,38"
2,446,Norfolk St @ Nelson St,100300,26
3,847,Lagrange St opp Virgil St,130200,"37,38"
4,3079,Adams St @ Minot St,100800,"201,19"


In [29]:
gdf_all = pd.merge(pd.merge(stops, mbta_stops), baseline_map)
gdf_all.head()

,STOP_ID,impacted_t,proportion,income,income_lev,impacted_p,geometry,STOP_NAME,TOWN,TOWN_ID,TRACT_ID,route_ids
0,3077,"416400,100800,100601,100700,100603","0.01,0.17,0.13,0.99,0.29",86562,2,1046,POINT (237120.669 892643.408),Gallivan Blvd @ opp Marsh St,BOSTON,35,100700,201
1,841,"130402,130300,130200,130100,130404","0.08,0.26,0.63,0.13,0.02",107359,2,632,POINT (227915.195 892644.017),Lagrange St @ Virgil Rd,BOSTON,35,130200,"37,38"
2,446,"100500,100400,100300,101102,101101,101002,1009...","0.17,0.18,1.00,0.41,0.06,0.24,0.02,1.00,0.37",46390,2,3056,POINT (234385.661 892644.944),Norfolk St @ Nelson St,BOSTON,35,100300,26
3,847,"130402,130300,130200,130100,130404","0.08,0.26,0.63,0.13,0.02",107359,2,632,POINT (227912.601 892650.156),Lagrange St opp Virgil St,BOSTON,35,130200,"37,38"
4,3079,"416400,100800,100601,100500,100400,100700,100603","0.00,0.51,0.32,0.01,0.03,0.67,0.06",83532,2,1647,POINT (236644.812 892651.990),Adams St @ Minot St,BOSTON,35,100800,"201,19"


In [31]:

gdf = gpd.GeoDataFrame(gdf_all, geometry='geometry')
gdf.to_file('./output/stops_complete.shp', driver='ESRI Shapefile')

In [32]:
gpd.read_file('./output/stops_complete_shapefile/stops_complete.shp')

,STOP_ID,impacted_t,proportion,income,income_lev,impacted_p,STOP_NAME,TOWN,TOWN_ID,TRACT_ID,route_ids,geometry
0,3077,"416400,100800,100601,100700,100603","0.01,0.17,0.13,0.99,0.29",86562,2,1046,Gallivan Blvd @ opp Marsh St,BOSTON,35,100700,201,POINT (237120.669 892643.408)
1,841,"130402,130300,130200,130100,130404","0.08,0.26,0.63,0.13,0.02",107359,2,632,Lagrange St @ Virgil Rd,BOSTON,35,130200,"37,38",POINT (227915.195 892644.017)
2,446,"100500,100400,100300,101102,101101,101002,1009...","0.17,0.18,1.00,0.41,0.06,0.24,0.02,1.00,0.37",46390,2,3056,Norfolk St @ Nelson St,BOSTON,35,100300,26,POINT (234385.661 892644.944)
3,847,"130402,130300,130200,130100,130404","0.08,0.26,0.63,0.13,0.02",107359,2,632,Lagrange St opp Virgil St,BOSTON,35,130200,"37,38",POINT (227912.601 892650.156)
4,3079,"416400,100800,100601,100500,100400,100700,100603","0.00,0.51,0.32,0.01,0.03,0.67,0.06",83532,2,1647,Adams St @ Minot St,BOSTON,35,100800,"201,19",POINT (236644.812 892651.990)
...,...,...,...,...,...,...,...,...,...,...,...,...
7805,9097,"336402,336401,336300,336200,336100","0.17,0.18,0.26,0.11,0.01",99903,2,639,Grove St @ Lebanon St,MELROSE,178,336300,131,POINT (236229.381 911541.866)
7806,5911,"336402,336401,336300,336200,336100","0.17,0.17,0.27,0.12,0.01",101026,2,644,Grove St @ Lebanon St,MELROSE,178,336300,131,POINT (236236.036 911542.538)
7807,5975,"337102,336402,336401,336300,336100","0.05,0.93,0.20,0.00,0.08",80781,2,1204,Wyoming Ave opp Cleveland St,MELROSE,178,336402,132,POINT (234977.387 911544.999)
7808,15976,"337102,336402,336401,336300,336100","0.05,0.93,0.20,0.00,0.08",80781,2,1204,Wyoming Ave @ Cleveland St,MELROSE,178,336402,132,POINT (234971.098 911547.184)
